In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from scipy.sparse import csc_matrix, coo_matrix
from sksparse.cholmod import cholesky
import random
import os
from scipy.stats import norm
from torch.distributions import MultivariateNormal
import matplotlib.pyplot as plt

# Create output directory
os.makedirs('training_visualizations', exist_ok=True)
EPSILON = 1e-9

class DeepNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(DeepNeuralNet, self).__init__()
        layers = []
        
        # Input to first hidden layer
        layers.append(nn.Linear(input_size, hidden_sizes[0]))
        layers.append(nn.ReLU())
        
        # Hidden layers with ReLU activations
        for i in range(len(hidden_sizes) - 1):
            layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
            layers.append(nn.ReLU())
        
        # Last hidden to output (no activation on output layer)
        layers.append(nn.Linear(hidden_sizes[-1], output_size))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

class SparseGaussian:
    def __init__(self, mean, covariance_sparse):
        self.mean = mean.cpu().numpy()
        self.covariance_sparse = covariance_sparse
        self.factor = cholesky(covariance_sparse)
        
    def sample(self, sample_shape):
        if isinstance(sample_shape, tuple):
            n_samples = sample_shape[0]
        else:
            n_samples = sample_shape
        
        z = np.random.randn(len(self.mean), n_samples)
        samples = self.factor.solve_A(z)
        samples = self.mean[:, np.newaxis] + samples
        return torch.from_numpy(samples.T).float()
    
    def log_prob(self, value):
        # Simplified log probability calculation
        value_np = value.cpu().numpy()
        centered = value_np - self.mean
        return torch.tensor(-0.5 * np.sum(centered**2))

def create_diagonal_mask(dimension):
    mask = []
    for i in range(dimension):
        mask.append((i, i))
    return mask

def update_mask(mask, dimension):
    n = dimension
    if len(mask) >= n * n:
        return mask
    while True:
        i, j = random.randint(0, n-1), random.randint(0, n-1)
        if (i, j) not in mask:
            mask.append((i, j))
            break
    return mask

def standard_gaussian(x):
    return (1./np.sqrt(2*np.pi)) * torch.exp(-x*x / 2.0)

def gaussian_cdf(x):
    return 0.5 * (1.0 + torch.erf(x * (1./np.sqrt(2))))

def relu_covariance(input_mean, input_covariance, std_of_neuron_variances):
    # Cov(ReLU(X_i), ReLU(X_j)) = E[ReLU(X_i) * ReLU(X_j)] - E[ReLU(X_i)] * E[ReLU(X_j)], Very hard bc ReLU clips negative vals
    input_covariance = input_covariance.tocoo()
    # Pre-compute values needed for all neurons (computed once, reused)
    normalized_mean = input_mean / std_of_neuron_variances
    gaussian_cdf_vals = gaussian_cdf(normalized_mean)
    standard_gaussian_vals = standard_gaussian(normalized_mean)
    rows, cols, data = [], [], []
    for idx in range(len(input_covariance.data)):
        i, j = input_covariance.row[idx], input_covariance.col[idx]
        input_cov_ij = input_covariance.data[idx]
        # Compute correlation coefficient matrix: correlations[i,j] = input_covariance[i,j] / (std_of_neuron_variances[i] * std_of_neuron_variances[j])
        correlation_coefficient = input_cov_ij / (std_of_neuron_variances[i] * std_of_neuron_variances[j])
        correlation_coefficient = torch.clamp(correlation_coefficient, -1/(1+EPSILON), 1/(1+EPSILON))  # Ensure valid correlations
        # 4-term series expansion for ReLU covariance
        t1 = correlation_coefficient * std_of_neuron_variances[i] * gaussian_cdf_vals[i] * \
             std_of_neuron_variances[j] * gaussian_cdf_vals[j]
        t2 = (1./2) * correlation_coefficient**2 * std_of_neuron_variances[i] * standard_gaussian_vals[i] * \
             std_of_neuron_variances[j] * standard_gaussian_vals[j]
        t3 = (1./6) * correlation_coefficient**3 * (-input_mean[i]) * standard_gaussian_vals[i] * \
             (-input_mean[j]) * standard_gaussian_vals[j]
        t4 = (1./24) * correlation_coefficient**4 * std_of_neuron_variances[i] * \
             (normalized_mean[i]**2 - 1) * standard_gaussian_vals[i] * \
             std_of_neuron_variances[j] * (normalized_mean[j]**2 - 1) * standard_gaussian_vals[j]
        output_cov_ij = t1 + t2 + t3 + t4
        rows.append(i)
        cols.append(j)
        data.append(float(output_cov_ij.detach().cpu().numpy()))
    n_features = input_covariance.shape[0]
    output_covariance_sparse = csc_matrix((data, (rows, cols)), shape=(n_features, n_features))
    return output_covariance_sparse

def propagate_gaussian_analytically(input_mean, input_covariance_sparse, layer, mask=None):
    # Add support for masks!
    def softrelu(x):
        # softrelu ≈ 0, softrelu ≈ μ/σ, or softrelu ≈ "partial activation"
        return standard_gaussian(x) + x*gaussian_cdf(x)
    
    rows, cols, data = [], [], []
    for i, j in mask:
        rows.append(i)
        cols.append(j)
        data.append(input_covariance_sparse[i, j])
    masked_covariance_sparse = csc_matrix((data, (rows, cols)), shape=input_covariance_sparse.shape)

    if isinstance(layer, nn.Linear):
        output_mean = layer(input_mean)
        W = layer.weight.detach().cpu().numpy()
        W_sparse = csc_matrix(W)
        temp = masked_covariance_sparse @ W_sparse.T 
        output_covariance_sparse = W_sparse @ temp
        
    elif isinstance(layer, nn.ReLU):
        input_shape = input_mean.shape
        # Flatten to vector and get stats
        input_mean_flat = input_mean.reshape(-1)

        # Extract diagonal variances from sparse matrix
        diagonal_indices = masked_covariance_sparse.diagonal()
        neuron_variances = torch.tensor(diagonal_indices, dtype=torch.float32)
        std_of_neuron_variances = torch.sqrt(neuron_variances)
        
        # ReLU(x) = max(0, x) is hard so use softrelu(x) = φ(x) + x*Φ(x) to propagate mean
        output_mean = std_of_neuron_variances * softrelu(input_mean_flat / std_of_neuron_variances)
        output_mean = output_mean.reshape(input_shape)
        
        # ReLU changes not just individual variances, but also correlations between neurons
        output_covariance_sparse = relu_covariance(input_mean_flat, masked_covariance_sparse, std_of_neuron_variances)
    
    return output_mean, output_covariance_sparse

def compute_estimator_mse(true_output_distribution, predicted_output_distribution):
    true_mean = torch.tensor(true_output_distribution.mean)
    predicted_mean = torch.tensor(predicted_output_distribution.mean)
    mse = torch.mean((true_mean - predicted_mean) ** 2)
    return mse.item()

def compute_estimator_kl_div(true_output_distribution, predicted_output_distribution):
    mu_true = true_output_distribution.mean
    mu_pred = predicted_output_distribution.mean
    sigma_true = true_output_distribution.covariance_sparse.toarray()
    sigma_pred = predicted_output_distribution.covariance_sparse.toarray()
    # Compute mean difference
    mu_diff = mu_pred - mu_true
    # Compute KL divergence: KL(P||Q) = 0.5 * [tr(Σ_Q^(-1) * Σ_P) + (μ_Q - μ_P)^T * Σ_Q^(-1) * (μ_Q - μ_P) - k + ln(det(Σ_Q)/det(Σ_P))]
    k = len(mu_true)
    sigma_true_inv = np.linalg.inv(sigma_true)
    trace_term = np.trace(sigma_true_inv @ sigma_pred)
    quad_term = mu_diff.T @ sigma_true_inv @ mu_diff
    log_det_term = np.log(np.linalg.det(sigma_true)) - np.log(np.linalg.det(sigma_pred))
    kl_div = 0.5 * (trace_term + quad_term - k + log_det_term)
    return kl_div

def find_true_output_distribution(input_distribution, model, n_samples=100000):
    model.eval()
    with torch.no_grad():
        input_samples = input_distribution.sample(torch.Size([n_samples]))
        output_samples = model(input_samples)
        output_samples_np = output_samples.cpu().numpy()
        empirical_mean = np.mean(output_samples_np, axis=0)
        empirical_cov = np.cov(output_samples_np, rowvar=False)
        empirical_cov_sparse = csc_matrix(empirical_cov)
        empirical_mean_tensor = torch.from_numpy(empirical_mean).float()
    return SparseGaussian(empirical_mean_tensor, empirical_cov_sparse)

def calculate_range_exceedance_probability(sparse_gaussian_dist, lower_bound=-5, upper_bound=5):
    mean = sparse_gaussian_dist.mean
    mu = mean[0] if len(mean.shape) > 0 else mean
    variance = sparse_gaussian_dist.covariance_sparse[0, 0]
    sigma = np.sqrt(max(variance, 1e-9))
    prob_below = norm.cdf(lower_bound, loc=mu, scale=sigma)
    prob_above = 1 - norm.cdf(upper_bound, loc=mu, scale=sigma)
    return prob_below + prob_above

def calculate_covariance_tracking_ratio(true_distribution, estimated_distribution):
    true_covariance_sparse = true_distribution.covariance_sparse
    estimated_covariance_sparse = estimated_distribution.covariance_sparse
    
    true_norm = np.sqrt((true_covariance_sparse.multiply(true_covariance_sparse)).sum())
    estimated_norm = np.sqrt((estimated_covariance_sparse.multiply(estimated_covariance_sparse)).sum())
    
    if true_norm == 0:
        return 1.0 if estimated_norm == 0 else 0.0
    
    return min(estimated_norm / true_norm, 1.0)

def calculate_total_covariance_entries(model, input_dim):
    total_entries = 0
    current_dim = input_dim
    for layer in model.network:
        if isinstance(layer, nn.Linear):
            total_entries += current_dim * current_dim
            current_dim = layer.out_features
        elif isinstance(layer, nn.ReLU):
        #     # For ReLU, we only consider the diagonal entries
            total_entries += current_dim * current_dim
    total_entries += current_dim * current_dim
    
    return total_entries

def calculate_initial_entries(model, input_dim):
    initial_entries = 0
    current_dim = input_dim
    for layer in model.network:
        if isinstance(layer, nn.Linear):
            initial_entries += current_dim
            current_dim = layer.out_features
    initial_entries += current_dim
    
    return initial_entries

def create_perfect_ordering(input_distribution, model, initial_masks):
    # Get true output distribution for comparison
    true_output_distribution = find_true_output_distribution(input_distribution, model)
    
    # Create working copies of masks, ensuring they are lists for append operations
    current_masks = [list(mask) for mask in initial_masks]
    optimal_ordering = []
    
    # Initial conditions
    initial_mean = input_distribution.mean
    initial_covariance = csc_matrix(input_distribution.covariance_matrix.numpy())
    
    # Pre-calculate layer dimensions to avoid recomputation
    layer_dimensions = []
    current_dim = input_distribution.mean.shape[0]
    
    for layer in model.network:
        print(layer, current_dim)
        if isinstance(layer, nn.Linear):
            layer_dimensions.append(current_dim)
            current_dim = layer.out_features
        elif isinstance(layer, nn.ReLU):
            layer_dimensions.append(current_dim)
    # Loop indefinitely until no more beneficial entries can be found
    while True:
        best_kl_divergence = float('inf')
        best_choice = None
        
        # Try adding entries to each layer
        for layer_idx, layer in enumerate(model.network):
           # if isinstance(layer, nn.Linear):
                current_mask = current_masks[layer_idx]
                dimension = layer_dimensions[layer_idx]
                
                # Try adding each possible new entry to this layer's mask
                for i in range(dimension):
                    for j in range(dimension):
                        if (i, j) not in current_mask:
                            # Create temporary masks with this new entry added
                            temp_masks = [mask.copy() for mask in current_masks]
                            temp_masks[layer_idx].append((i, j))
                            
                            # Simulate forward propagation with these temporary masks
                            try:
                                temp_mean = initial_mean
                                temp_covariance = initial_covariance
                                
                                for temp_layer_idx, temp_layer in enumerate(model.network):
                                    temp_mean, temp_covariance = propagate_gaussian_analytically(
                                        temp_mean, temp_covariance, temp_layer, temp_masks[temp_layer_idx]
                                    )
                                
                                # Evaluate this choice by computing KL divergence
                                estimated_output_distribution = SparseGaussian(temp_mean.detach(), temp_covariance)
                                kl_divergence = compute_estimator_kl_div(true_output_distribution, estimated_output_distribution)
                                
                                # If this choice is the best so far, record it
                                if kl_divergence < best_kl_divergence:
                                    best_kl_divergence = kl_divergence
                                    best_choice = (layer_idx, i, j)
                                    
                            except Exception as e:
                                # This choice resulted in an error, so skip it.
                                # For debugging, you might want to uncomment the following line:
                                print(f"Skipping choice ({layer_idx}, {i}, {j}) due to error: {e}")
                                continue
            # else:
            #     # If the layer is not Linear, we skip it for mask additions
            #     continue

        
        # After checking all possible additions, update the ordering with the best one found
        if best_choice is not None:
            layer_idx, i, j = best_choice
            optimal_ordering.append(best_choice)
            current_masks[layer_idx].append((i, j))
            
            if len(optimal_ordering) % 20 == 0:
                print(f"  Step {len(optimal_ordering)}: Added ({i}, {j}) to layer {layer_idx}, KL = {best_kl_divergence:.6f}")
        else:
            # If best_choice is still None, no valid entry could be added.
            # This means the process is complete.
            print(f"No more beneficial or valid entries found after {len(optimal_ordering)} additions. Terminating.")
            break # Exit the while loop

    print(f"Optimal ordering computed with {len(optimal_ordering)} entries.")
    return optimal_ordering

def update_mask_with_ordering(masks, ordering, step):
    if step < len(ordering):
        layer_idx, i, j = ordering[step]
        if (i, j) not in masks[layer_idx]:
            masks[layer_idx].append((i, j))
        if layer_idx != 0 and layer_idx % 2 == 0:
            step = step + 1
            masks[layer_idx-1] = masks[layer_idx]
    return masks, step

torch.manual_seed(42)
input_mean = torch.zeros(3)
full_covariance_matrix = torch.tensor([
    [1.0, 0.7, 0.3],
    [0.7, 1.5, 0.8],
    [0.3, 0.8, 2.0]
])
input_distribution = MultivariateNormal(loc=input_mean, covariance_matrix=full_covariance_matrix)
X = input_distribution.sample(torch.Size([1000]))
true_weights = torch.tensor([[2.0], [3.0], [-1.0]])
y = X @ true_weights

# --- Model Training ---
model = DeepNeuralNet(input_size=3, hidden_sizes=[4, 12, 4], output_size=1)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

model.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

# Print final model architecture
print("\nModel Architecture:")
print(model)


# --- Main Propagation Loop with Perfect Ordering ---
input_dim = 3 
total_entries = calculate_total_covariance_entries(model, input_dim)
initial_entries = calculate_initial_entries(model, input_dim)
print(f"Total possible covariance entries across all layers: {total_entries}")
print(f"Initial diagonal entries: {initial_entries}")
initial_mean = input_distribution.mean
initial_covariance = csc_matrix(input_distribution.covariance_matrix.numpy())
training_steps = 5000
masks = []
current_dim = input_dim
for layer_idx, layer in enumerate(model.network):
    if isinstance(layer, nn.Linear):
        masks.append(create_diagonal_mask(current_dim))
        current_dim = layer.out_features
    elif isinstance(layer, nn.ReLU):
        masks.append(create_diagonal_mask(current_dim))


# PRE-COMPUTE THE OPTIMAL ORDERING
print("Computing optimal ordering (this may take a while)...")
optimal_ordering = create_perfect_ordering(input_distribution, model, masks)
print(f"Optimal ordering contains {len(optimal_ordering)} entries")
training_steps = min(training_steps, len(optimal_ordering))
print(f"Running {training_steps} training steps")
kl_divergences = []
estimated_exceedance_probs = []
covariance_tracking_ratios = []
percent_filled = [] 
updated_step = 0
true_output_distribution = find_true_output_distribution(input_distribution, model)
true_exceedance_prob = calculate_range_exceedance_probability(true_output_distribution)

with torch.no_grad():
    for step in range(training_steps):
        updated_step += 1
        propagated_gaussians = []
        current_mean = initial_mean
        current_covariance = initial_covariance
        if updated_step > 0:
            masks, updated_step = update_mask_with_ordering(masks, optimal_ordering, step - 1)
        
        # Calculate current percentage filled
        current_entries = initial_entries + updated_step
        current_percent = (current_entries / total_entries) * 100
        percent_filled.append(current_percent)
        for layer_idx, layer in enumerate(model.network):
            output_mean, output_covariance = propagate_gaussian_analytically(current_mean, current_covariance, layer, masks[layer_idx])
            propagated_layer_gaussian = SparseGaussian(output_mean, output_covariance)
            propagated_gaussians.append(propagated_layer_gaussian)
            current_mean = output_mean
            current_covariance = output_covariance
        estimated_output_distribution = propagated_gaussians[-1]

        # Calculate metrics for this step
        estimator_mse = compute_estimator_mse(true_output_distribution, estimated_output_distribution)
        kl_divergence = compute_estimator_kl_div(true_output_distribution, estimated_output_distribution)
        estimated_exceedance_prob = calculate_range_exceedance_probability(estimated_output_distribution)
        covariance_tracking_ratio = calculate_covariance_tracking_ratio(true_output_distribution, estimated_output_distribution)
        kl_divergences.append(kl_divergence)
        estimated_exceedance_probs.append(estimated_exceedance_prob)
        covariance_tracking_ratios.append(covariance_tracking_ratio)
        if updated_step % 100 == 0:
            next_addition = optimal_ordering[step] if step < len(optimal_ordering) else "None"
            print(f"Step {step}: {current_percent:.2f}% filled, KL Div = {kl_divergence:.6f}, Est. Exceedance = {estimated_exceedance_prob:.6f}/{true_exceedance_prob:.6f}, Cov. Ratio = {covariance_tracking_ratio:.6f}, Next: {next_addition}")

# --- Create Visualization ---
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))
ax1.plot(percent_filled, kl_divergences, 'b-', linewidth=2, label='KL Divergence')
ax1.set_xlabel('Covariance Matrix Filled (%)')
ax1.set_ylabel('KL Divergence')
ax1.set_title('KL Divergence vs Covariance Matrix Fill Percentage')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax2.plot(percent_filled, estimated_exceedance_probs, 'r-', linewidth=2, label='Estimated Exceedance Probability')
ax2.axhline(y=true_exceedance_prob, color='g', linestyle='--', linewidth=2, label='True Exceedance Probability')
ax2.set_xlabel('Covariance Matrix Filled (%)')
ax2.set_ylabel('Exceedance Probability')
ax2.set_title('Exceedance Probability (Outside [-5, 5]) vs Covariance Matrix Fill Percentage')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax3.plot(percent_filled, covariance_tracking_ratios, 'm-', linewidth=2, label='Covariance Tracking Ratio')
ax3.axhline(y=1.0, color='k', linestyle='--', linewidth=1, alpha=0.7, label='Perfect Tracking (1.0)')
ax3.set_xlabel('Covariance Matrix Filled (%)')
ax3.set_ylabel('Covariance Tracking Ratio')
ax3.set_title('Covariance Tracking Ratio vs Covariance Matrix Fill Percentage')
ax3.grid(True, alpha=0.3)
ax3.legend()
ax3.set_ylim(0, 1.1)
plt.tight_layout()
plt.savefig('training_visualizations/gaussian_propagation_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

# --- Output ---
print("\nFinal Results:")
print(f"Final Percentage Filled: {percent_filled[-1]:.2f}%")
print(f"Final KL Divergence: {kl_divergences[-1]:.6f}")
print(f"Final Estimated Exceedance Probability: {estimated_exceedance_probs[-1]:.6f}")
print(f"True Exceedance Probability: {true_exceedance_prob:.6f}")
print(f"Difference in Exceedance Probabilities: {abs(estimated_exceedance_probs[-1] - true_exceedance_prob):.6f}")
print(f"Final Covariance Tracking Ratio: {covariance_tracking_ratios[-1]:.6f}")

Epoch 0, Loss: 22.6110
Epoch 20, Loss: 22.5852
Epoch 40, Loss: 22.4629
Epoch 60, Loss: 21.6043
Epoch 80, Loss: 12.8311

Model Architecture:
DeepNeuralNet(
  (network): Sequential(
    (0): Linear(in_features=3, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=12, bias=True)
    (3): ReLU()
    (4): Linear(in_features=12, out_features=4, bias=True)
    (5): ReLU()
    (6): Linear(in_features=4, out_features=1, bias=True)
  )
)
Total possible covariance entries across all layers: 362
Initial diagonal entries: 24
Computing optimal ordering (this may take a while)...
Linear(in_features=3, out_features=4, bias=True) 3
ReLU() 4
Linear(in_features=4, out_features=12, bias=True) 4
ReLU() 12
Linear(in_features=12, out_features=4, bias=True) 12
ReLU() 4
Linear(in_features=4, out_features=1, bias=True) 4
  Step 20: Added (3, 1) to layer 2, KL = 0.284682
  Step 40: Added (0, 11) to layer 4, KL = 0.283059
  Step 60: Added (2, 5) to layer 3, KL = 0.283053
  Step

/Users/noahschwartz/Desktop/Estimating_NNs/.venv/lib/python3.11/site-packages/numpy/linalg/_linalg.py:2430: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


  Step 220: Added (10, 9) to layer 4, KL = 0.069539
  Step 240: Added (11, 9) to layer 3, KL = 0.066482
  Step 260: Added (3, 0) to layer 6, KL = 0.063582
  Step 280: Added (2, 0) to layer 4, KL = 0.063586
  Step 300: Added (9, 4) to layer 4, KL = 0.063782
No more beneficial or valid entries found after 318 additions. Terminating.
Optimal ordering computed with 318 entries.
Optimal ordering contains 318 entries
Running 318 training steps
Step 1: 6.63% filled, KL Div = 0.599239, Est. Exceedance = 0.000144/0.126254, Cov. Ratio = 0.126523, Next: (0, 1, 0)
Step 200: 61.88% filled, KL Div = 0.106682, Est. Exceedance = 0.059274/0.126254, Cov. Ratio = 0.494401, Next: (3, 9, 10)
Step 201: 61.88% filled, KL Div = 0.103791, Est. Exceedance = 0.060682/0.126254, Cov. Ratio = 0.500475, Next: (4, 9, 10)


CholmodNotPositiveDefiniteError: /private/tmp/suite-sparse-20250505-6185-gw9cj5/SuiteSparse-7.10.3/CHOLMOD/Cholesky/t_cholmod_rowfac_worker.c:433: not positive definite (code 1)